![banner](../logo/banner.png)

# 3.5 Sensores Digitales

La simulación de sensores digitales básicos es muy sencillo aplicando la configuración pull-up o pull-down. Dado que la señal de un alto o bajo representa un estado u otro.

## 3.5.1 Sensor PIR HC-SR501 (Sensor de presencia - Sensor Infrarrojo pasivo)

Este sensor nos manda una señal alta de 3.3V cuando existe presencia o recibe radiación infrarroja. Se puede alimentar entre 5V a 12V. [Datasheet](https://www.allelectronics.com/mas_assets/media/allelectronics2018/spec/PIR-7.pdf)

![sensor PIR](imgs/pir_down.jpeg)

### 3.5.1.1 Aplicando el Sensor PIR

**Ejemplo 1:**

Tenemos un sensor PIR, que cuando exista presencia mande a activiar un parpadeo de LEDs de forma rápida (medio segundo), mientras no exista presencia los LEDs deben parpadear de manera lenta (2seg)

- Circuito 

![circuito](imgs/circuito_pir.png)

- Simulación

![simulacion](imgs/circuito_pir_simulacion.png)

**Ejemplo 2:**

Ahora tenemos un sensor de presencia junto a un switch en una habitación. Lo que queremos es que si existe presencia y el switch se presiona se encienda la luz del cuarto, pero si la persona deja la habitación pero no apago la luz se debe apagar el foco en un tiempo definido. Al salir del cuarto se presiona el switch se debe apagar el foco.

![pir 2](imgs/pir2_simul.png)

```python
from pyfirmata import Arduino, util
from time import sleep

def control_led(board,pin,value):
    board.digital[pin].write(value)
    

PORT = 'COM#'
# importamos los elementos necesarios
board = Arduino(PORT)

it = util.Iterator(board)  # le pasamos la tarjeta al iterador
it.start()  # arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
PIR_INPUT = 2
SWICTH_INPUT = 3
board.get_pin(f'd:{str(PIR_INPUT)}:i')
board.get_pin(f'd:{str(SWICTH_INPUT)}:i')

# declaro mi lista de la posicion de los leds
FOCO = 8

status_foco = False

count = 0 # contador del temporizador

while True:

    pir = board.digital[PIR_INPUT].read()
    switch = board.digital[SWICTH_INPUT].read()

    if pir and switch and not status_foco :
        control_led(board, FOCO, 1)
        status_foco = True
        sleep(0.25) # estabilizamos el presionar el boton
    elif switch and status_foco:
        control_led(board, FOCO, 0)
        status_foco = False
        sleep(1)

    # temporizador para apagar el foco en caso de que no exista presencia
    if status_foco and not pir:
        if count >= 10:
            control_led(board, FOCO, 0)
            status_foco = False
            count = 0
        sleep(1)
        count += 1
    else:
        count = 0
```

## 3.5.2 Sensor de Flama

El sensor de flama esta constituido por un fototransistor acoplado a comparador de voltaje.

Da una señal de 5V a su salida, indicando que esta recibiendo la radiación del fuego.

![flama](imgs/flame-sensor.jpg)

Aplicaciones:

- Hydrogen stations
- Industrial heating
- Fire detection
- Fire alarm
- Fire fighting robot
- Drying systems
- Industrial gas turbines
- Domestic heating systems
- Gas-powered cooking devices

El circuito del sensor de flama

![circuito sensor de flama](imgs/sensor_flama_circuit.png)

### 3.5.2.1 Aplicando el Sensor de flama

**Ejemplo 1:**

Tenemos un sensor de flama que cuando detecte fuego este debe mandar una señal auditiva.

![flame circuit](imgs/flame_circuit.png)

![flama simu](imgs/flama1_simul.png)

**Ejemplo 2:**

Tenemos un sensor PIR, un sensor de flama, estos sensores están en la cocina. Si el sensor de flama dectecta fuego y no existe presencia, este sensor se coloca en dirección de las hornillas; debe mandar abrir la valvula solenoide (representada con un LED apagado) colocada en el tanque de gas y activara un extractor de aire (representado por un motor DC ). Pero si existe presencia y existe flama o no, la solenoide debe estar activa y el extractor apagado. Si esta encendido el extractor y no vuelve la persona a la cocina, se debe apagar despues de un cierto tiempo.

![flama_2](imgs/flama2_simu.png)

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
# importamos los elementos necesarios
board = Arduino(PORT)

def control_output(board,pin,value):
    board.digital[pin].write(value)

it = util.Iterator(board)  # le pasamos la tarjeta al iterador
it.start()  # arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
FLAME_INPUT = 2
PIR_INPUT = 3

board.get_pin(f'd:{str(FLAME_INPUT)}:i')
board.get_pin(f'd:{str(PIR_INPUT)}:i')

# declaro mis salidas
SOLENOIDE = 8
EXTRACTOR = 9

status_extractor = False
count_time_extractor = 0

while True:

    flame = board.digital[FLAME_INPUT].read()
    pir = board.digital[PIR_INPUT].read()

    if flame and not pir:
        control_output(board, SOLENOIDE, 0)
        control_output(board, EXTRACTOR, 1)
        status_extractor =True
        sleep(0.25)
    elif pir:
        control_output(board, SOLENOIDE, 1)
        control_output(board, EXTRACTOR, 0)
        status_extractor = False
        sleep(0.25)

    if status_extractor:
        if count_time_extractor == 10:
            control_output(board, EXTRACTOR, 0)
            status_extractor = False
            count_time_extractor = 0

        count_time_extractor += 1
        sleep(1)
```

## 3.5.3 Sensor Magnético (Switch magnético)

Este tipo de sensor reacciona a campos magnéticos, al colocar cerca hara que los contactos del switch cierre, haciendo que cierre el circuito.

![sensor](imgs/switch_magnetico.jpg)

Otro tipo de switch, a este se le tiene que complementar con electronica, es el siguiente:

![](imgs/Sensor-Magnetico-Switch-MC-38.png)

### 3.5.3.1 Aplicando el Sensor de magnético

**Ejemplo 1:**

Tenemos un sensor de magnético en una ventana. Cuando la ventana sea abra parpadee un LED, mientras esta cerrada la ventana que el LED este apagado.

![sensor magnetico simulacion](imgs/magentico_simul.png)

```python
from pyfirmata import Arduino, util
from time import sleep

def secuencia(board,leds=[],time=0.1):
    for led in leds:
        board.digital[led].write(1)
    sleep(time)
    for led in leds:
        board.digital[led].write(0)
    sleep(time)

PORT = 'COM#'
# importamos los elementos necesarios
board = Arduino(PORT)

it = util.Iterator(board)  # le pasamos la tarjeta al iterador
it.start()  # arrancamos a el iterador para poder leer entradas

# configuro el pin como ENTRADA
MAGENTICO_INPUT = 2
board.get_pin(f'd:{str(MAGENTICO_INPUT)}:i')

# declaro en donde esta mi alarma
ALARM = [8]

while True:

    magnetico = board.digital[MAGENTICO_INPUT].read()

    if not magnetico:
        print('ventana abierta')
        secuencia(board=board,leds=ALARM,time=0.5)
```

## 3.5.4 Sensor de Efecto Hall (Sensor de corriente)

Este sensor es capaz de captar el flujo de la corriente electrica, es decir, capta el campo generado electromagnetico que genera la corriente al pasar por un conductor.

Es decir, si detacta corriente sobre un conductor o se le acerca un imán este mandará la señal alta (1 lógico)

![hall](imgs/hall_1.jpeg)

## 3.5.5 Sensor de Luz

Este sensor 

![sensor luz](imgs/sensor_luz.png)

## 3.5.6 Sensor Infrarrojo (Como evasor de obtaculos o de proximidad)

Este sensor tiene muchas aplicaciones y esto depende de la forma en como se coloque y la interpretación que queramos o necesitemos. *Todos los que se muestran funcionan igual*, lo que cambia es la posición del emisor y receptor.

![infrarrojo1](imgs/proximidad.jpg)

![infrarrojo2](imgs/sensor-infrarrojo-.webp)

![infrarrojo3](imgs/infrarojo2.jpeg)

---
## Tarea - Salidas digitales

1. Simula un sensor de luz el cual haga encender un foco cuando incida poca luz (por debajo del 45%). En caso que supere esa cantidad de luz el foco se debe mantener apagado.
2. Simula un sensor de distancia (infrarrojo). Carrito seguido de luz. Se tendran 2 sensores infrarrojos, el derecho e izquierdo, cuando el sensor derecho detecte luz debe arrancar una llanta (motor DC izquierdo), cuando el sensor izquierdo reciba luz debe encender la llanta derecha (motor DC derecho). Cuando no reciba luz ningun sensor debe apagar ambos motores, si ambos reciben luz deben encender ambos motores.

---

Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)

[Mecatrónica 85](https://mecatronica85.com/)

[fb/mecatronica85](https://www.facebook.com/mecatronica85)